# Tutorial 23: gvar and lsqfit

## PHYS 2600, Fall 2022

In [ ]:
# Import cell
%pip install -q gvar lsqfit
import numpy as np
import matplotlib.pyplot as plt
import gvar as gv
import lsqfit

## T23.1 - Basics of gvar

### Part A

Let's start with just making `gvar`s and doing simple arithmetic on them.  Enter the following values as `gvar` objects:

* x = 2.19(52)
* y = 0.93(13)

Try using both the string and numeric mean/error versions of `gv.gvar()`.

In [ ]:
#

Now, calculate the following quantities and print them out as `gvar`s:

* $w = x+y$
* $z = xy$

In [ ]:
#

### Part B

Next, verify that the propagation of error matches what you would expect from the analytic formulas:

$$
\sigma_{w} = \sqrt{\sigma_x^2 + \sigma_y^2} \\
\frac{\sigma_{z}}{z} = \sqrt{\frac{\sigma_x^2}{x^2} + \frac{\sigma_y^2}{y^2}}
$$

In order to calculate these formulas, you'll need to access the mean values and error bars individually from `x` and `y`.  To do this, you can use the `.mean` or `.sdev` properties, or the `gv.mean()` or `gv.sdev()` functions.

(In other words: if `g` is a `gvar` object, either `g.mean` or `gv.mean(g)` will give you its mean value as a number.)

In [ ]:
# Checking error propagation
print(w.sdev)
print(np.sqrt(x.sdev**2 + y.sdev**2))

print(z.sdev)
print(z.mean * np.sqrt( (x.sdev / x.mean)**2 + (y.sdev / y.mean)**2 ) )

Use `gv.corr()` to find the _correlation coefficient_ between `w` and `z` - it should be non-zero since they were both produced from the same Gaussian variables, `x` and `y`!

Calculate the ratio `w/z`.  Then create two _brand new_ `gvar` variables, `w_new` and `z_new`, which have the same mean and error values as `w` and `z`.  Show that `w_new/z_new` has a much larger error bar than `w/z`, due to the missing correlations.

In [ ]:
#

### Part C

Sometimes, it's more convenient to build many `gvar` objects at once from separate lists of means and errors.  To do this, we just pass matching lists to the `gv.gvar()` function:

In [ ]:
some_means = [3.2,5.4,3.7,1.2,1.7]
some_errors = [1.9,0.4,1.1,0.5,0.9]

some_gv = gv.gvar(some_means, some_errors)
print(some_gv)

Notice that this automatically gives us an array of `gvars`.  One nice thing about `gvar` is that it plays very nicely with `numpy`, so we can do all the usual array operations when we have lists like this.

First, a warm-up: use NumPy to __compute the mean of `some_gv`__.  (You should find `3.04(49)`.)

In [ ]:
#

We can also use masks with arrays of `gvar` objects.  In the cell below, __create a mask__ to select all of the elements of `some_gv` which satisfy $\mu / \sigma > 2$ - i.e., anything whose central value is at least twice as large as its error bar.  (Another way to state this is that we are selecting point with "_signal-to-noise ratio_" SNR of at least 2.)

_(Hint: `gv.mean()` and `gv.sdev()` are happy to act on entire arrays of `gvar` objects at once, returning a matching array of mean values or error values.)_

In [ ]:
#

You should see the following print-out:

```
[5.40(40) 3.7(1.1) 1.20(50)]
```

## T23.2 - Getting started with `lsqfit`

As usual, there are a lot of available modules that can help us carry out least-squares modeling in Python - for example, the `scipy.optimize.curve_fit` function can do general non-linear least squares.

We're going to learn a different module called `lsqfit` - you can find [the documentation here](https://lsqfit.readthedocs.io/en/latest/).  The `lsqfit` module is something I use in research often: it is more general and powerful than the `scipy` curve fitter, and it works natively with the `gvar` module (both are from the same author.)

To warm up, we'll start with a simple linear fit to some real data taken from a teaching photoelectric effect experiment.  Here are the data points:

| f [$10^{14}$ Hz] | $V_{\rm stop}$ [V] |
|-------------------------|--------------------|
| 5.490 | 0.24(4) |
| 6.879 | 0.80(10) |
| 7.408 | 1.20(20) |
| 8.213 | 1.50(25) |

I've also provided the data in the cell below, along with a quick function that makes an errorbar plot of gvar data.  __Run the cell to load the data variables and make a plot.__

In [ ]:
f_data = np.array([5.490, 6.879, 7.408, 8.213]) # 10^{14} Hz
Vcut = gv.gvar(['0.24(4)', '0.80(10)', '1.20(20)', '1.50(25)']) # V

def plot_gvar(x, y_gv):
    """
    Given two arrays of equal length, one of which contains
    gvar objects, returns an errorbar plot of y(x).
    
    Arguments:
    ====
    x: Array of x-values.
    y_gv: Array of y-values with error [as gvars.]
    
    Returns:
    =====
    An errorbar plot.
    """
    
    return plt.errorbar(x, gv.mean(y_gv), yerr=gv.sdev(y_gv), linestyle='', marker='o')
    
    
plot_gvar(f_data, Vcut)
plt.xlim(4.5, 9.0)
plt.ylim(0,2)
plt.xlabel('f [$\\times 10^{14}$ Hz]')
plt.ylabel('$V_{\\rm stop} [V]$')

We expect a linear relationship between the light frequency $f$ and the stopping voltage $V_{\rm stop}$ - looks plausible, although the data are a bit noisy.  To test the linear model and get the work function and Planck's constant, we need to set up a fit!

### Part A

The first thing we'll need is a model function.  It turns out that `lsqfit` wants this to be implemented as a Python function, although one nice feature of `lsqfit` is that we can use a dictionary to keep our parameter names straight.  __Implement the function `PE_model(f, a)` below__, where `f` should be an array of frequency values and `a` is a dictionary of fit parameters.  

You should assume that `a` has two keys called `Phi` (capital-phi, written $\Phi$) and `h`, which are the two variables in the model function we want:

$$
eV_{\rm stop} = hf - \Phi.
$$

The function should return the value of the right-hand side, $hf - \Phi$.  (We'll be fitting this against the $eV_{\rm stop}$ data; what you're returning is the _prediction_ of the model for $eV_{\rm stop}$.)

In [ ]:
def PE_model(f, a):    
    #


In [ ]:
import numpy.testing as npt

npt.assert_allclose(PE_model(5, {'h': 1, 'Phi': 1}), 4)
npt.assert_allclose(PE_model(f_data, {'h': 0, 'Phi': 0}), [0, 0, 0, 0])
npt.assert_allclose(PE_model(f_data, {'h': 1, 'Phi': 1}), [4.49, 5.879, 6.408, 7.213])

Now use your function `PE_model` to __make a plot showing the data__ (copy from the cell above) __as well as the model__, using a guess for the fit parameters of `(h, Phi) = (0.5, 3)` (which I came up with by staring at the plot - maybe you can do better?)

In [ ]:
a0 = {
    'h': 0.5,
    'Phi': 3,
}

#

### Part B

Now let's actually run the fitter and find the best fit!  The main workhorse function of `lsqfit` is called `lsqfit.nonlinear_fit()`.  (Yes, even though this is a linear fit, we can use this function - linear fits are a special case so it will still work.)  The documentation for `lsqfit.nonlinear_fit()` is pretty complicated, because it has a lot of features.  But recalling the definition of $\chi^2$,

$$
\chi^2 = \sum_i \left(\frac{y_i - f(x_i, \mathbf{a})}{\sigma_i} \right)^2,
$$

we expect that `lsqfit.nonlinear_fit` must take the following arguments at least:

* Data for independent variable(s), $x_i$ - this should be __a NumPy array__.
* Data for dependent variable(s), $y_i$, and error bars $\sigma_i$ - this should be a __`gvar` array__.
* The model function $f$ - we wrote this in part A.

It will also need an __initial guess__, $\mathbf{a}_0$, because the methods it uses to minimize the $\chi^2$ function require a starting point.  (Having a good initial guess can often greatly improve the speed and accuracy of a nonlinear fit!)  

Once we have our data `x` and `y`, model function `f`, and initial guess `a0`, the call signature is:

```
fit_result = lsqfit.nonlinear_fit(data=(x,y), fcn=f, p0=a0)
```

This will run the fit and return an object containing lots of information.  

__Call the `nonlinear_fit` function below__, using the variables you defined above and the initial guess `a0` that you plotted just above.  Then __print the object returned by `nonlinear_fit`__ (I like to call it "`fit_result`") to see a report on the best fit!

In [ ]:
# fit_result = 
print(fit_result)

There's lots of information to unpack there.  The most important features in the report are:

* At the top, the reduced $\chi^2 / N_{\rm dof}$, following by $N_{\rm dof}$ in square brackets.  Finally, the quantity $Q$ is the p-value of the fit: the probability that we would draw the given experimental data if the best-fit model is true.  (This should be a good fit: reduced $\chi^2$ should be smaller than 1, and $Q$ should be pretty large.)
* Under the heading "Parameters:" on the left, the best-fit value with error bar for each parameter.  On the right, the initial values of each parameter.  (The "error bar" on our initial guess is infinity, because our guess had no error.  If we had __prior information__ about one or more parameters, we could impose that as a prior constraint on the fit, one of `lsqfit`'s advanced features.)

The information under "Settings:" mostly has to do with the internal workings of the fitter, and can be safely ignored unless you are running into numerical problems with doing the fit itself.

Note that although calling `print(fit_result)` gives us a nice text-formatted report, `fit_result` itself is actually an object that contains other information in the form of properties (accessed with `.` notation.)  In the cell below, __try accessing the following properties__:

* `fit_result.chi2`: Un-reduced value of $\chi^2$ at best-fit point.
* `fit_result.dof`: The number of degrees of freedom.
* `fit_result.p`: Dictionary of best-fit parameters as gvars.
* `fit_result.x` and `fit_result.y`: The data used in the fit.  (Often useful to use later!)

In [ ]:
#

### Part C

Now, a little more advanced analysis.  When we do a fit, even if the chi-squared seems good (but definitely if it isn't), a good practice is to look at the __fit residuals__.  These are defined as

$$
r_i = y_i - f(x_i, \mathbf{a}^\star)
$$

where $\mathbf{a}^\star$ are the best-fit parameters.  (You can also try to normalize by the error bars $y_i$, which you can access using `gv.sdev()`.  This makes it possible to read off the "chi-squared-by-eye" contribution from each point - but the plot will look similar without the normalization.)


In [ ]:
resids = (fit_result.y - PE_model(fit_result.x, fit_result.p)) / gv.sdev(fit_result.y)
print(resids)
plot_gvar(f_data, resids)

Hopefully, you will see that the points are scattered around zero with no obvious systematic pattern - a good sign that our model is indeed successfully describing the data.

Next, __convert your fit results to an estimate for Planck's constant__.  By working in volts we've been implicitly dividing everything by the electron charge $e = 1.602 \times 10^{-19}$ C, so you'll need to multiply the parameters through by that.  Don't forget to account for the factor of $10^{14}$ we pulled out of the frequency data!  The real value is

$$
h = 6.626 \times 10^{-34}\ \rm{J} \cdot \rm{s}
$$

How close is your result?  Is the more precise value of $h$ within the error bar of your estimate?

Using `fit_result.p`, calculate the ratio of your two model parameters, i.e. calculate $\Phi / h$.  Not worrying about units or the $10^{14}$, if I go back to the fit itself and take the ratio with standard error propagation, I find `4.96(94)`.  But if you use `fit_result.p`, __the error on the ratio should be much smaller.__ Why?

In [ ]:
#

Finally, implement the two alternative models, constant and quadratic:

$$
eV_{\rm stop} = C \\
eV_{\rm stop} = kf^2 - \Phi
$$

Run the nonlinear fits for both cases against the same data.  How do the p-values ("Q"-values) compare to the linear model?  Can you reject either or both alternative models based on this data set?

In [ ]:
#

### Part D (optional challenge) - fitting to exponential decay

The exercise above was a nice introduction, but linear fits are really easy: they're safe from a lot of the problems that can appear in more general non-linear fits.  Let's step up the difficulty and try to fit an _exponential decay_.

The cell below reads in a sequence of random decays - actually, they're generated using the Monte Carlo for radioactive decay from the homework.  The data consists of time (in seconds) and a set of particle counts from repeated trials.  (We'll see how to read data files next week - this is a little preview!)

As a reminder, our model for the expected number of particles remaining is:

$$
N(t) = N_0 e^{-t/\tau}.
$$

We have a partial sample of the data, running from 10 to 25 seconds, so we'll need to fit to determine both the size of the initial sample $N_0$ and the lifetime $\tau$.

First, __run the cell below__ to import the data and plot it.  (The data file isn't _too_ complicated, but it might be time-consuming to set up so I've provided the code.)

In [ ]:
import os
import sys
import urllib.request

remote_path = "https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/tutorials/tut23/decay_one_dataset.csv"
local_dir = "/content/"
# local_dir = "./" # uncomment this line if you aren't running in Colab
local_path = f"{local_dir}decay_one_dataset.csv"

if not os.path.exists(local_path):
    urllib.request.urlretrieve(remote_path, local_path)

if local_dir not in sys.path:
    sys.path.append(local_dir)
    
ds = np.genfromtxt('decay_one_dataset.csv')
t_data = ds[:,0]
decay_data = gv.dataset.avg_data(ds[:,1:].T)
print(decay_data)
plot_gvar(t_data, decay_data)

Now write a fit model function, then __run the nonlinear fit__ with initial guess $500$ for the initial abundance and $0.1$ for the lifetime.

In [ ]:
def fcn_exp_one(t, a):
    #

You should find that your model is an extremely poor description of the data!  This doesn't mean your model function is wrong; it's actually a symptom of numerical instability, which exponential functions are notorious for.  __Try the fit again in the cell below__, changing the initial guess to 10.

In [ ]:
#

This is from our model of nitrogen-16, so the correct lifetime is 10.286 seconds and the correct initial sample size was 1000 particles.  Hopefully your fit now agrees with those values and has a good p-value!